In [1]:
import boto3
import os

In [2]:
#variables
OHIO_REGION = "us-east-2"
NORTH_VIRGINIA = "us-east-1"
AMI_UBUNTU_OHIO = "ami-020db2c14939a8efb"
AMI_UBUNTU_NORTH_VIRGINIA = "ami-0279c3b3186e54acd"
KEY_PAIR_OHIO_NAME = "key_pair_marcelo_ohio"
KEY_PAIR_NORTH_VIRGINIA_NAME = "key_pair_marcelo_north_virginia"

In [3]:
#create a key_pair
def create_key_pair(client,key_name:str):
    key_pair = client.create_key_pair(KeyName=key_name)

    private_key = key_pair["KeyMaterial"]

    # write private key to file with 400 permissions
    print(f"Writing {key_name}.pem in directory ./")
    with os.fdopen(os.open(f"./{key_name}.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)
    return key_pair

#delete a key_pair
def delete_key_pair(client,key_name:str):
    key_pair = client.delete_key_pair(KeyName=key_name)
    print(f"removing {key_name} if exist")
    # write private key to file with 400 permissions
    try:
        os.remove("./" + key_name + ".pem")
        
    except:
        pass
def create_an_instance(client, ami,key_name):
    instancetype = "t2.micro"
    print(f"Creating an instance with {instancetype}, and keyname {key_name['KeyName']}")
    instances = client.run_instances(
        ImageId=ami,
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName=key_name['KeyName']
    )
    for i in instances['Instances']:
        if i['KeyName'] == key_name["KeyName"]:
            instance_id = i['InstanceId']
    return instances, instance_id

def delete_instances(client,key_pair):
    list_instances_id = []
    for i in client.describe_instances(Filters=[
        {
            'Name': 'key-name',
            'Values': [
                key_pair,
            ]
        },
        {
            'Name': 'instance-state-name',
            'Values': [
                "pending","running","stopping","stopped"
            ]
        },
    ],
)['Reservations']:
        list_instances_id.append(i['Instances'][0]["InstanceId"])
    if len(list_instances_id) > 0:
        print(f"Deleting Instances ids {list_instances_id}")
        waiter = client.get_waiter('instance_terminated')
        client.terminate_instances(InstanceIds=list_instances_id)
        waiter.wait(InstanceIds=list_instances_id)
    else:
        print(f"There is no instances not terminated with the keypair {key_pair}")

In [4]:
#checking if boto3 client is working
iam = boto3.client("iam")

for user in iam.list_users()["Users"]:
    if user["UserName"] == "marcelo":
        print(user["UserName"])
        print(user["UserId"])
        print(user["Arn"])
        print(user["CreateDate"])

marcelo
AIDATTPQFU3W27CNBJLEK
arn:aws:iam::248001177325:user/marcelo
2021-09-21 18:22:34+00:00


In [5]:
#connection to ec2
ec2_ohio = boto3.client('ec2', region_name='us-east-2')
ec2_north_virginia_ = boto3.client('ec2', region_name='us-east-1')

In [11]:
delete_instances(ec2_ohio,KEY_PAIR_OHIO_NAME)
delete_instances(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

Deleting Instances ids ['i-07fb4c7387df9a284']
Deleting Instances ids ['i-09ab2bee88a0fe818']


In [7]:
delete_key_pair(ec2_ohio,KEY_PAIR_OHIO_NAME)
delete_key_pair(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

removing key_pair_marcelo_ohio if exist
removing key_pair_marcelo_north_virginia if exist


In [8]:
key_pair_ohio = create_key_pair(ec2_ohio,KEY_PAIR_OHIO_NAME)
key_pair_north_virginia = create_key_pair(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

Writing key_pair_marcelo_ohio.pem in directory ./
Writing key_pair_marcelo_north_virginia.pem in directory ./


In [9]:
instances_ohio, instance_ohio_id = create_an_instance(ec2_ohio,AMI_UBUNTU_OHIO,key_pair_ohio)
instances_north_virginia, instance_north_virginia_id = create_an_instance(ec2_north_virginia_,AMI_UBUNTU_NORTH_VIRGINIA,key_pair_north_virginia)

Creating an instance with t2.micro, and keyname key_pair_marcelo_ohio
Creating an instance with t2.micro, and keyname key_pair_marcelo_north_virginia


In [10]:
ec2_north_virginia_.describe_instances(Filters=[
        {
            'Name': 'key-name',
            'Values': [
                KEY_PAIR_NORTH_VIRGINIA_NAME,
            ]
        },
        {
            'Name': 'instance-state-name',
            'Values': [
                "pending","running","stopping","stopped"
            ]
        },
    ],
)['Reservations']

[{'Groups': [],
  'Instances': [{'AmiLaunchIndex': 0,
    'ImageId': 'ami-0279c3b3186e54acd',
    'InstanceId': 'i-09ab2bee88a0fe818',
    'InstanceType': 't2.micro',
    'KeyName': 'key_pair_marcelo_north_virginia',
    'LaunchTime': datetime.datetime(2021, 11, 19, 14, 41, 45, tzinfo=tzutc()),
    'Monitoring': {'State': 'disabled'},
    'Placement': {'AvailabilityZone': 'us-east-1a',
     'GroupName': '',
     'Tenancy': 'default'},
    'PrivateDnsName': 'ip-172-31-95-237.ec2.internal',
    'PrivateIpAddress': '172.31.95.237',
    'ProductCodes': [],
    'PublicDnsName': '',
    'State': {'Code': 0, 'Name': 'pending'},
    'StateTransitionReason': '',
    'SubnetId': 'subnet-8a2f91a4',
    'VpcId': 'vpc-5d9e5327',
    'Architecture': 'x86_64',
    'BlockDeviceMappings': [],
    'ClientToken': '5955a824-2579-4bdf-9df2-746a8c3e3db7',
    'EbsOptimized': False,
    'EnaSupport': True,
    'Hypervisor': 'xen',
    'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2021